## Code to generate manifest.csv
- Based on this [schema](https://www.neurobagel.org/documentation/mr_proc/configs/)

### QPN naming convention
- `visit`: timepoint of **ANY** clinical (i.e. UPDRS, MoCA, Neuropsy) data collected 
    - e.g. V01, V02 etc. 
- `session`: timepoint of MRI collection
    - e.g. ses-01, ses-02 etc.
- `event`: timepoint relative to a consensus baseline (used for inter-modality i.e. MRI vs clinical data harmonization) 
    - e.g. baseline, m06, m12 etc. 

In [1]:
import pandas as pd
import numpy as np

### Paths

In [2]:
releases_dir = "/home/nikhil/projects/Parkinsons/qpn/releases/"

previous_release = "June_2023"
current_release = "Sept_2023"

# Previous mr_proc manifest
previous_manifest_csv = f"{releases_dir}{previous_release}/tabular/manifest.csv"

# Current mr_proc manifest
current_manifest_csv = f"{releases_dir}{current_release}/tabular/manifest.csv"

# Current recruit manifest
current_recruit_manifest_xls = f"{releases_dir}{current_release}/tabular/Suivi_RPQ.xlsx"

### Read recruitment manifest from previous release

In [3]:
previous_recruit_manifest_df = pd.read_csv(previous_manifest_csv)
previous_recruit_manifest_df["participant_id"] = previous_recruit_manifest_df["participant_id"].str.strip()
nipoppy_participants_previous = previous_recruit_manifest_df["participant_id"].dropna().unique()
n_nipoppy_participants_previous = len(nipoppy_participants_previous)

print(f"number of participants from previous mr_proc release: {n_nipoppy_participants_previous}")
previous_recruit_manifest_df.head()

number of participants from previous mr_proc release: 207


,participant_id,visit,session,datatype
0,MNI0056,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']"
1,MNI0058,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']"
2,MNI0068,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']"
3,MNI0079,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']"
4,MNI0103,V01,ses-01,"['anat', 'dwi', 'fmap', 'func']"


### Read latest recruitment manifest

In [4]:
col_range = "A:N"

col_rename_dict = {"Unnamed: 0":"participant_id",
                "IRM01\n(J-M-A)":"IRM01_date", "#IRM 1\n PD":"IRM01_PD", "#IRM 1\n CTRL":"IRM01_CTRL", "# IRM 1\n RBD":"IRM01_RBD",
                "IRM 2 \n(J-M-A)":"IRM02_date", "#IRM 2\n PD":"IRM02_PD", "#IRM 2\n CTRL":"IRM02_CTRL", "# IRM 2\n RBD":"IRM02_RBD"}

useful_cols = col_rename_dict.values()

suivi_df = pd.read_excel(current_recruit_manifest_xls,sheet_name="En cours", engine='openpyxl', usecols=col_range)
suivi_df = suivi_df.rename(columns=col_rename_dict)[useful_cols].copy()

# remove the row with tally
suivi_df = suivi_df.drop([0])

# remove rows without participant_id
suivi_df = suivi_df.dropna(axis=0, subset=["participant_id"])
suivi_df = suivi_df[~suivi_df["participant_id"].astype(str).isin(["0"])] 

# remove subjects without imaging data
suivi_df = suivi_df[(suivi_df["IRM01_PD"] == 1) | (suivi_df["IRM01_CTRL"] == 1) | (suivi_df["IRM01_RBD"] == 1) | 
                    (suivi_df["IRM02_PD"] == 1) | (suivi_df["IRM02_CTRL"] == 1) | (suivi_df["IRM02_RBD"] == 1)]


# fix participant_id formatting issues
# Some rows have Dx in participant_id and one participant with two IDs with "="
possible_delimiters = [" ", "(", "="]
for delim in possible_delimiters:
    suivi_df["participant_id"] = suivi_df["participant_id"].str.split(delim, 1, expand=True)[0]

# nipoppy_participants_current
nipoppy_participants_current = suivi_df["participant_id"].dropna().unique()

suivi_df

,participant_id,IRM01_date,IRM01_PD,IRM01_CTRL,IRM01_RBD,IRM02_date,IRM02_PD,IRM02_CTRL,IRM02_RBD
15,PD00016,26-07-2019,1.0,0.0,0.0,0,NaN,NaN,NaN
19,PD00020,2018-05-12 00:00:00,1.0,0.0,0.0,24-11-2022,1.0,0.0,0.0
31,PD00032,24-07-2019,1.0,0.0,0.0,0,NaN,NaN,NaN
47,PD00048,21-08-2019,1.0,0.0,0.0,0,NaN,NaN,NaN
117,PD00119,13-08-2018,1.0,0.0,0.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2000,MNI0452,2023-04-08 00:00:00,1.0,0.0,0.0,0,NaN,NaN,NaN
2001,MNI0453,2023-11-08 00:00:00,0.0,1.0,0.0,0,NaN,NaN,NaN
2024,MNI0476,2023-12-09 00:00:00,1.0,0.0,0.0,0,NaN,NaN,NaN
2153,MNI0521,16-08-2023,0.0,1.0,0.0,0,NaN,NaN,NaN


### Set date columns and check visit order

In [5]:
# set date columns to datetime
# has mixed types, but auto formatting + coerce works fine here
suivi_df["IRM01_date"] = pd.to_datetime(suivi_df["IRM01_date"], errors="coerce")
suivi_df["IRM02_date"] = pd.to_datetime(suivi_df["IRM02_date"], errors="coerce")

# Check visit orders
suivi_df["visit_interval (V2-V1) in days"] = suivi_df["IRM02_date"] - suivi_df["IRM01_date"]
suivi_df["visit_interval (V2-V1) in days"] = suivi_df["visit_interval (V2-V1) in days"].dt.days
visits_wit_wrong_order_df = suivi_df[suivi_df["visit_interval (V2-V1) in days"] < 0]
print(f"Participants with wrong visit order: ({len(visits_wit_wrong_order_df)}) :{visits_wit_wrong_order_df['participant_id'].values}")


Participants with wrong visit order: (0) :[]


### Get new participants per session
This is based on a valid date for MRI visit

In [6]:
total_partcipants_additions = list(set(nipoppy_participants_current) - set(nipoppy_participants_previous))
print(f"number of new participants: {len(total_partcipants_additions)}")

MRI_v1_participants = suivi_df[(suivi_df["IRM01_PD"] == 1) | (suivi_df["IRM01_CTRL"] == 1) | (suivi_df["IRM01_RBD"] == 1)]["participant_id"].dropna().unique()
MRI_v2_participants = suivi_df[(suivi_df["IRM02_PD"] == 1) | (suivi_df["IRM02_CTRL"] == 1) | (suivi_df["IRM02_RBD"] == 1)]["participant_id"].dropna().unique()

visit_participant_dict = {"MRI_v1": MRI_v1_participants, "MRI_v2": MRI_v2_participants}

print(f"MRI_v1_participants: {len(MRI_v1_participants)}, MRI_v2_participants: {len(MRI_v2_participants)}")

number of new participants: 61
MRI_v1_participants: 265, MRI_v2_participants: 48


### Generate current_manifest_df

- Populate manifest with available clinical visits / MRI sessions
- Datatypes: Assuming QPN has all 4 BIDS datatypes: ["anat","dwi","fmap","func"]

- Sample `manifest.csv`

| participant_id | participant_dicom_dir | visit | session | datatype                     | bids_id |
|----------------|-----------------------|-------|---------|------------------------------|---------|
| 001            | MyStudy_001_V2021      | V01   | ses-01  | ["anat","dwi","fmap","func"] | sub-001 |
| 001            | MyStudy_001_V2022      | V02   | ses-02  | ["anat"]                     | sub-001 |
| 002            | MyStudy_002_V2021      | V01   | ses-01  | ["anat","dwi"]               | sub-002 |
| 002            | MyStudy_002_V2024      | V03   | ses-03  | ["anat","dwi"]               | sub-002 |

In [7]:
visit_labels = visit_participant_dict.keys()
visit_session_dict = {"MRI_v1": "ses-01", "MRI_v2": "ses-02"}

manifest_cols = ["visit","session","datatype"]
avail_datatypes = "['anat','dwi','fmap','func']"

current_manifest_df = pd.DataFrame()

for visit_label in visit_labels:
    visit_participant_ids = visit_participant_dict[visit_label]
    print(f"visit_id: {visit_label}, n_participants: {len(visit_participant_ids)}")

    _df = pd.DataFrame(index=visit_participant_ids, columns=manifest_cols)

    session = visit_session_dict[visit_label]

    _df.loc[visit_participant_ids,"visit"] = visit_label
    _df.loc[visit_participant_ids,"session"] = session
    _df.loc[visit_participant_ids,"datatype"] = avail_datatypes
    
    current_manifest_df = current_manifest_df.append(_df)

current_manifest_df

visit_id: MRI_v1, n_participants: 265
visit_id: MRI_v2, n_participants: 48


,visit,session,datatype
PD00016,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00020,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00032,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00048,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00119,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
...,...,...,...
MNI0206,MRI_v2,ses-02,"['anat','dwi','fmap','func']"
MNI0215,MRI_v2,ses-02,"['anat','dwi','fmap','func']"
MNI0259,MRI_v2,ses-02,"['anat','dwi','fmap','func']"
MNI0324,MRI_v2,ses-02,"['anat','dwi','fmap','func']"


### Save update CSV

In [8]:
save_current_manifest = False
if save_current_manifest:
    print(f"Saving new nipoppy manifest here: {current_manifest_csv}")
    current_manifest_df = current_manifest_df.reset_index().rename(columns={"index":"participant_id"})
    current_manifest_df.to_csv(current_manifest_csv,index=None)